In [535]:
import pandas as pd
import random
import uuid
import os

# Data Grooming

## Statline Data

### Load Historical Data

In [536]:
statline_path = 'Historical Data/Team Statlines/Yearly/'
dir_list = os.listdir(statline_path)

statline_dfs = list()
for file_name in dir_list:
    statline_df = pd.read_csv(statline_path + file_name)
    statline_dfs.append(statline_df)
    
statline_df = pd.concat(statline_dfs)
statline_df = statline_df.drop_duplicates()
statline_df['Date'] = pd.to_datetime(statline_df['Date'], format='mixed')
statline_df = statline_df.sort_values(by='Date').reset_index(drop=True)
statline_df['Obs UUID'] = statline_df.apply(lambda r: str(uuid.uuid4()), axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,OREB,TmRb,REB,AST,STL,BLK,TO,F,Tm Pts,Obs UUID
0,Richmond,"W, 44-42",2007-11-05,UME,N,200,44,0.663,65,17,...,6,0,18,9,13,6,18,10,0,a81c9171-eaf4-4ecc-b2bf-cae87dd82b0d
1,Maine,"L, 42-44",2007-11-05,RICH,N,200,42,0.652,65,19,...,11,0,39,9,9,1,28,12,0,e93a5edf-b807-49dc-ad18-8a1a0dd87e4f
2,Tennessee-Martin,"L, 71-102",2007-11-05,MEM,V,200,71,0.785,90,24,...,15,0,41,16,5,2,18,22,0,e1141379-f50a-4202-bd46-86c5039cdf57
3,Memphis,"W, 102-71",2007-11-05,UTM,H,200,102,1.149,90,37,...,18,0,52,23,9,11,10,21,0,f07a9cef-54f9-4ffe-bd6b-7456a599b59b
4,Alabama A&M,"L, 55-69",2007-11-06,GWB,N,200,55,0.702,77,17,...,10,0,30,9,4,8,27,24,0,480f90ae-2f81-4997-aebf-7043d5e3899e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,12,0,35,8,3,2,8,17,0,ca8a0f1b-573e-4cd9-8c35-236ca2fb5bdf
171967,Florida Atlantic,"L, 71-72",2023-04-01,SDSU,V,196,71,1.110,64,23,...,7,0,31,6,6,2,9,17,0,fb606a53-2805-47a1-9a67-495d1c6fb9f6
171968,Miami (Fla.),"L, 59-72",2023-04-01,CONN,V,195,59,0.912,64,20,...,12,0,29,10,8,1,9,12,0,92e25ac8-5047-4c0d-97dd-31b930e236e7
171969,San Diego State,"L, 59-76",2023-04-03,CONN,V,193,59,0.861,68,19,...,12,0,33,7,7,2,12,20,0,5a711232-0bde-4dc6-a4ea-cc7b3131cbb4


### Team Abbreviation and Nickname Data

In [537]:
team_df = pd.read_csv('Historical Data/ncaamTeams.csv', index_col=0)
team_df = team_df.drop_duplicates()
team_df

,ID,Name,Abbrev,Active,Nickname
team_376,376.0,Akron Zips,AKR,Y,Zips
team_344,344.0,Alabama Crimson Tide,ALA,Y,Crimson Tide
team_545,545.0,Alabama A&M Bulldogs,ALAM,Y,Bulldogs
team_238,238.0,Alabama State Hornets,ALST,Y,Hornets
team_540,540.0,Albany Great Danes,ALB,Y,Great Danes
...,...,...,...,...,...
NaN,NaN,Lindenwood Lions,LIND,Y,Lions
NaN,NaN,Queens (N.C.) Royals,QUNC,Y,Royals
NaN,NaN,Southern Indiana Screaming Eagles,SIND,Y,Screaming Eagles
NaN,NaN,Stonehill Skyhawks,STNH,Y,Skyhawks


Make sure each team's full name ends with their nickname

In [538]:
team_df['Correct'] = team_df.apply(lambda row: row['Name'].endswith(row['Nickname']), axis=1)
team_df.loc[team_df['Correct'] == False]

,ID,Name,Abbrev,Active,Nickname,Correct


The College of William & Mary (CWM) is often mislabeled as WMU (Western Michigan), similarly, Northwestern (NW) is often mislabeled as NU (Northeastern).  We are ignoring those abbreviations for the time being and will handle them later.

In [539]:
abbrev_to_name = {row['Abbrev']: row['Name'].replace(row['Nickname'], '').strip() for index, row in team_df.iterrows()}
del abbrev_to_name['WMU']
del abbrev_to_name['NU']

### Rename Opponents

In [540]:
statline_df['Opp'] = statline_df.apply(lambda row: abbrev_to_name.get(row['Opp'], row['Opp']), axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,OREB,TmRb,REB,AST,STL,BLK,TO,F,Tm Pts,Obs UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,6,0,18,9,13,6,18,10,0,a81c9171-eaf4-4ecc-b2bf-cae87dd82b0d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,0,39,9,9,1,28,12,0,e93a5edf-b807-49dc-ad18-8a1a0dd87e4f
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,15,0,41,16,5,2,18,22,0,e1141379-f50a-4202-bd46-86c5039cdf57
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,18,0,52,23,9,11,10,21,0,f07a9cef-54f9-4ffe-bd6b-7456a599b59b
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,10,0,30,9,4,8,27,24,0,480f90ae-2f81-4997-aebf-7043d5e3899e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,Florida Atlantic,H,195,72,1.135,64,25,...,12,0,35,8,3,2,8,17,0,ca8a0f1b-573e-4cd9-8c35-236ca2fb5bdf
171967,Florida Atlantic,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,7,0,31,6,6,2,9,17,0,fb606a53-2805-47a1-9a67-495d1c6fb9f6
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,12,0,29,10,8,1,9,12,0,92e25ac8-5047-4c0d-97dd-31b930e236e7
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,12,0,33,7,7,2,12,20,0,5a711232-0bde-4dc6-a4ea-cc7b3131cbb4


In [541]:
shortened_names = {'Virginia Commonwealth': 'VCU',
                   'Mount St. Mary`s': 'Mount Saint Mary`s',
                   'Florida Atlantic': 'FAU',
                   'Maryland-Baltimore County': 'UMBC',
                   'Sam Houston': 'Sam Houston State',
                   'Texas Christian': 'TCU'}

statline_df['Team'] = statline_df.apply(lambda row: shortened_names.get(row['Team'], row['Team']), axis=1)
statline_df['Opp'] = statline_df.apply(lambda row: shortened_names.get(row['Opp'], row['Opp']), axis=1)

statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,OREB,TmRb,REB,AST,STL,BLK,TO,F,Tm Pts,Obs UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,6,0,18,9,13,6,18,10,0,a81c9171-eaf4-4ecc-b2bf-cae87dd82b0d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,0,39,9,9,1,28,12,0,e93a5edf-b807-49dc-ad18-8a1a0dd87e4f
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,15,0,41,16,5,2,18,22,0,e1141379-f50a-4202-bd46-86c5039cdf57
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,18,0,52,23,9,11,10,21,0,f07a9cef-54f9-4ffe-bd6b-7456a599b59b
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,10,0,30,9,4,8,27,24,0,480f90ae-2f81-4997-aebf-7043d5e3899e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,12,0,35,8,3,2,8,17,0,ca8a0f1b-573e-4cd9-8c35-236ca2fb5bdf
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,7,0,31,6,6,2,9,17,0,fb606a53-2805-47a1-9a67-495d1c6fb9f6
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,12,0,29,10,8,1,9,12,0,92e25ac8-5047-4c0d-97dd-31b930e236e7
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,12,0,33,7,7,2,12,20,0,5a711232-0bde-4dc6-a4ea-cc7b3131cbb4


### Remove "ND1" Opponents and Useless Columns

A large percentage of opponents are listed as "ND1" the assumption is that this is short for Non-D1 opponent since the frequency is too high for any team. Since there is no mapping or data for ND1, these are removed

In [542]:
statline_df = statline_df.loc[statline_df['Opp'] != 'ND1']
statline_df = statline_df.drop(columns=['Tm Pts'])
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,FTA,OREB,TmRb,REB,AST,STL,BLK,TO,F,Obs UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,7,6,0,18,9,13,6,18,10,a81c9171-eaf4-4ecc-b2bf-cae87dd82b0d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,5,11,0,39,9,9,1,28,12,e93a5edf-b807-49dc-ad18-8a1a0dd87e4f
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,22,15,0,41,16,5,2,18,22,e1141379-f50a-4202-bd46-86c5039cdf57
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,29,18,0,52,23,9,11,10,21,f07a9cef-54f9-4ffe-bd6b-7456a599b59b
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,26,10,0,30,9,4,8,27,24,480f90ae-2f81-4997-aebf-7043d5e3899e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,22,12,0,35,8,3,2,8,17,ca8a0f1b-573e-4cd9-8c35-236ca2fb5bdf
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,21,7,0,31,6,6,2,9,17,fb606a53-2805-47a1-9a67-495d1c6fb9f6
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,12,12,0,29,10,8,1,9,12,92e25ac8-5047-4c0d-97dd-31b930e236e7
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,20,12,0,33,7,7,2,12,20,5a711232-0bde-4dc6-a4ea-cc7b3131cbb4


### Parse Results, Date

In [543]:
statline_df['Victory'] = statline_df.apply(lambda r: 1 if r['Res'][0] == 'W' else 0, axis=1)
statline_df['Year'] = statline_df.apply(lambda r: r['Date'].year, axis=1)
statline_df['Month'] = statline_df.apply(lambda r: r['Date'].month, axis=1)
statline_df['Day'] = statline_df.apply(lambda r: r['Date'].day, axis=1)
statline_df['Result Points'] = statline_df.apply(lambda r: int(r['Res'].split()[1].split('-')[0]), axis=1)
statline_df['Result Points Allowed'] = statline_df.apply(lambda r: int(r['Res'].split()[1].split('-')[1]), axis=1)
statline_df['Season'] = statline_df.apply(lambda r: r['Year'] if r['Month'] < 6 else r['Year'] + 1, axis=1)
statline_df['Calculated Possessions'] = statline_df.apply(lambda r: r['FGA'] - r['OREB'] + r['TO'] + int(r['FTA'] / 2), axis=1)
statline_df['Points Per Possession'] = statline_df.apply(lambda r: 0 if r['Calculated Possessions'] == 0 else r['PTS'] / r['Calculated Possessions'], axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Obs UUID,Victory,Year,Month,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,a81c9171-eaf4-4ecc-b2bf-cae87dd82b0d,1,2007,11,5,44,42,2008,66,0.666667
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,e93a5edf-b807-49dc-ad18-8a1a0dd87e4f,0,2007,11,5,42,44,2008,64,0.656250
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,e1141379-f50a-4202-bd46-86c5039cdf57,0,2007,11,5,71,102,2008,91,0.780220
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,f07a9cef-54f9-4ffe-bd6b-7456a599b59b,1,2007,11,5,102,71,2008,89,1.146067
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,480f90ae-2f81-4997-aebf-7043d5e3899e,0,2007,11,6,55,69,2008,79,0.696203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,ca8a0f1b-573e-4cd9-8c35-236ca2fb5bdf,1,2023,4,1,72,71,2023,64,1.125000
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,fb606a53-2805-47a1-9a67-495d1c6fb9f6,0,2023,4,1,71,72,2023,64,1.109375
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,92e25ac8-5047-4c0d-97dd-31b930e236e7,0,2023,4,1,59,72,2023,65,0.907692
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,5a711232-0bde-4dc6-a4ea-cc7b3131cbb4,0,2023,4,3,59,76,2023,69,0.855072


### Parse Location

In [544]:
statline_df['Location'] = statline_df['Loc'].map({'H': 'Home',
                                                  'V': 'Away',
                                                  'N': 'Neutral'})
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Victory,Year,Month,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,1,2007,11,5,44,42,2008,66,0.666667,Neutral
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,0,2007,11,5,42,44,2008,64,0.656250,Neutral
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,0,2007,11,5,71,102,2008,91,0.780220,Away
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,1,2007,11,5,102,71,2008,89,1.146067,Home
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,0,2007,11,6,55,69,2008,79,0.696203,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,1,2023,4,1,72,71,2023,64,1.125000,Home
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,0,2023,4,1,71,72,2023,64,1.109375,Away
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,0,2023,4,1,59,72,2023,65,0.907692,Away
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,0,2023,4,3,59,76,2023,69,0.855072,Away


### 2 Point FGs

In [545]:
statline_df['2FM'] = statline_df.apply(lambda r: r['FGM'] - r['3FM'], axis=1)
statline_df['2FA'] = statline_df.apply(lambda r: r['FGA'] - r['3FA'], axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,11,5,44,42,2008,66,0.666667,Neutral,13,30
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,5,42,44,2008,64,0.656250,Neutral,18,34
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,11,5,71,102,2008,91,0.780220,Away,14,40
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,11,5,102,71,2008,89,1.146067,Home,26,48
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,11,6,55,69,2008,79,0.696203,Neutral,13,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,4,1,72,71,2023,64,1.125000,Home,16,39
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,4,1,71,72,2023,64,1.109375,Away,14,30
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,4,1,59,72,2023,65,0.907692,Away,13,42
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,4,3,59,76,2023,69,0.855072,Away,13,36


In [546]:
statline_df['Calculated Points'] = statline_df.apply(lambda r: r['3FM'] * 3 + r['2FM'] * 2 + r['FTM'], axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,5,44,42,2008,66,0.666667,Neutral,13,30,44
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,5,42,44,2008,64,0.656250,Neutral,18,34,42
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,5,71,102,2008,91,0.780220,Away,14,40,71
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,5,102,71,2008,89,1.146067,Home,26,48,102
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,6,55,69,2008,79,0.696203,Neutral,13,31,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,1,72,71,2023,64,1.125000,Home,16,39,72
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,1,71,72,2023,64,1.109375,Away,14,30,71
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,1,59,72,2023,65,0.907692,Away,13,42,59
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,3,59,76,2023,69,0.855072,Away,13,36,59


### Error Checking

In [547]:
statline_df.loc[statline_df['PTS'] != statline_df['Result Points']]

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points
72,Jacksonville State,"L, 65-107",2007-11-09,Georgia,V,182,58,0.827,77,23,...,9,65,107,2008,70,0.828571,Away,14,45,58
168,Nebraska,"W, 67-52",2007-11-10,Presbyterian,H,222,69,1.177,58,26,...,10,67,52,2008,59,1.169492,Home,18,24,69
516,Northwestern State,"L, 66-72",2007-11-14,New Orleans,H,216,68,1.079,62,24,...,14,66,72,2008,63,1.079365,Home,17,36,68
529,Washington,"W, 81-77",2007-11-14,Utah,H,200,83,1.228,66,26,...,14,81,77,2008,68,1.220588,Home,22,50,83
1880,Cal State Northridge,"W, 69-43",2007-12-01,North Carolina Central,V,211,72,0.890,80,26,...,1,69,43,2008,81,0.888889,Away,22,42,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167849,Louisiana-Monroe,"L, 45-58",2023-01-28,James Madison,V,150,37,0.815,40,15,...,28,45,58,2023,45,0.822222,Away,11,20,37
167857,James Madison,"W, 58-45",2023-01-28,Louisiana-Monroe,H,96,37,1.095,40,13,...,28,58,45,2023,34,1.088235,Home,7,13,37
167941,Richmond,"L, 60-86",2023-01-28,Dayton,V,182,55,0.861,56,24,...,28,60,86,2023,64,0.859375,Away,20,39,55
169102,Michigan,"L, 61-62",2023-02-11,Indiana,H,132,41,0.859,60,15,...,11,61,62,2023,48,0.854167,Home,11,22,41


In [548]:
statline_df.loc[statline_df['PTS'] != statline_df['Calculated Points']]

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points


In [549]:
statline_df.loc[statline_df['Calculated Points'] != statline_df['Result Points']]

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Result Points,Result Points Allowed,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points
72,Jacksonville State,"L, 65-107",2007-11-09,Georgia,V,182,58,0.827,77,23,...,9,65,107,2008,70,0.828571,Away,14,45,58
168,Nebraska,"W, 67-52",2007-11-10,Presbyterian,H,222,69,1.177,58,26,...,10,67,52,2008,59,1.169492,Home,18,24,69
516,Northwestern State,"L, 66-72",2007-11-14,New Orleans,H,216,68,1.079,62,24,...,14,66,72,2008,63,1.079365,Home,17,36,68
529,Washington,"W, 81-77",2007-11-14,Utah,H,200,83,1.228,66,26,...,14,81,77,2008,68,1.220588,Home,22,50,83
1880,Cal State Northridge,"W, 69-43",2007-12-01,North Carolina Central,V,211,72,0.890,80,26,...,1,69,43,2008,81,0.888889,Away,22,42,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167849,Louisiana-Monroe,"L, 45-58",2023-01-28,James Madison,V,150,37,0.815,40,15,...,28,45,58,2023,45,0.822222,Away,11,20,37
167857,James Madison,"W, 58-45",2023-01-28,Louisiana-Monroe,H,96,37,1.095,40,13,...,28,58,45,2023,34,1.088235,Home,7,13,37
167941,Richmond,"L, 60-86",2023-01-28,Dayton,V,182,55,0.861,56,24,...,28,60,86,2023,64,0.859375,Away,20,39,55
169102,Michigan,"L, 61-62",2023-02-11,Indiana,H,132,41,0.859,60,15,...,11,61,62,2023,48,0.854167,Home,11,22,41


Result Points is wrong

In [550]:
statline_df = statline_df.drop(columns=['Result Points', 'Result Points Allowed'])
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Year,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,2007,11,5,2008,66,0.666667,Neutral,13,30,44
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,2007,11,5,2008,64,0.656250,Neutral,18,34,42
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,2007,11,5,2008,91,0.780220,Away,14,40,71
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,2007,11,5,2008,89,1.146067,Home,26,48,102
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,2007,11,6,2008,79,0.696203,Neutral,13,31,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,2023,4,1,2023,64,1.125000,Home,16,39,72
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,2023,4,1,2023,64,1.109375,Away,14,30,71
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,2023,4,1,2023,65,0.907692,Away,13,42,59
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,2023,4,3,2023,69,0.855072,Away,13,36,59


### Fix Incorrect Abbreviations for Northwestern and William & Mary

The mislabeling of Northwestern/Northeastern and William & Mary/ Western Michigan is handled here

In [551]:
northwestern_games = statline_df.loc[statline_df['Team'] == 'Northwestern']
for index, row in northwestern_games.iterrows():
    opponent = row['Opp']
    date = row['Date']
    matches = statline_df.loc[((statline_df['Team'] == opponent) | (statline_df['Team'] == 'William & Mary') | (statline_df['Team'] == 'Western Michigan')) & 
                              (statline_df['Opp'] == 'NU') & 
                              (statline_df['Date'] == date)]
    for matching_index, matching_row in matches.iterrows():
        statline_df.at[matching_index, 'Opp'] = 'Northwestern'

william_and_mary_games = statline_df.loc[statline_df['Team'] == 'William & Mary']
for index, row in william_and_mary_games.iterrows():
    opponent = row['Opp']
    date = row['Date']
    matches = statline_df.loc[((statline_df['Team'] == opponent) | (statline_df['Team'] == 'Northeastern') | (statline_df['Team'] == 'Northwestern')) &
                              (statline_df['Opp'] == 'WMU') &
                              (statline_df['Date'] == date)]
    for matching_index, matching_row in matches.iterrows():
        statline_df.at[matching_index, 'Opp'] = 'William & Mary'

statline_df['Opp'] = statline_df.apply(lambda r: 'Northeastern' if r['Opp'] == 'NU' else r['Opp'], axis=1)
statline_df['Opp'] = statline_df.apply(lambda r: 'Western Michigan' if r['Opp'] == 'WMU' else r['Opp'], axis=1)

statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Year,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,2007,11,5,2008,66,0.666667,Neutral,13,30,44
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,2007,11,5,2008,64,0.656250,Neutral,18,34,42
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,2007,11,5,2008,91,0.780220,Away,14,40,71
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,2007,11,5,2008,89,1.146067,Home,26,48,102
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,2007,11,6,2008,79,0.696203,Neutral,13,31,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,2023,4,1,2023,64,1.125000,Home,16,39,72
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,2023,4,1,2023,64,1.109375,Away,14,30,71
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,2023,4,1,2023,65,0.907692,Away,13,42,59
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,2023,4,3,2023,69,0.855072,Away,13,36,59


### Add UUID for Each Game

In [552]:
def get_loser_uuid(row):
    if row['Victory'] == 1:
        return row['UUID']

    team = row['Team']
    opp = row['Opp']
    date = row['Date']
    
    counterpart = statline_df.loc[(statline_df['Team'] == opp) &
                                  (statline_df['Opp'] == team) &
                                  (statline_df['Date'] == date)]

    if len(counterpart) != 1:
        return row['UUID']

    counterpart_uuid = counterpart.squeeze()['UUID']
    return counterpart_uuid

In [553]:
statline_df['UUID'] = statline_df.apply(lambda r: str(uuid.uuid4()) if r['Victory'] == 1 else None, axis=1)
statline_df['UUID'] = statline_df.apply(lambda r: get_loser_uuid(r), axis=1)
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,11,5,2008,66,0.666667,Neutral,13,30,44,01dbf56d-681b-47e1-917a-af746d6f906d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,5,2008,64,0.656250,Neutral,18,34,42,01dbf56d-681b-47e1-917a-af746d6f906d
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,11,5,2008,91,0.780220,Away,14,40,71,3a1dcea9-5dc4-4468-af40-bbdeb727914b
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,11,5,2008,89,1.146067,Home,26,48,102,3a1dcea9-5dc4-4468-af40-bbdeb727914b
4,Alabama A&M,"L, 55-69",2007-11-06,Gardner-Webb,N,200,55,0.702,77,17,...,11,6,2008,79,0.696203,Neutral,13,31,55,a83307d5-9410-475f-b95a-f054f5cb2111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171966,San Diego State,"W, 72-71",2023-04-01,FAU,H,195,72,1.135,64,25,...,4,1,2023,64,1.125000,Home,16,39,72,62792664-cb9d-46f6-a0ae-da5b4eac020a
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,4,1,2023,64,1.109375,Away,14,30,71,62792664-cb9d-46f6-a0ae-da5b4eac020a
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,4,1,2023,65,0.907692,Away,13,42,59,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,4,3,2023,69,0.855072,Away,13,36,59,9ba510b2-bd9b-4180-9602-0042dc6bd324


In [554]:
statline_df = statline_df.sort_values(by=['Date', 'UUID'], kind='mergesort')
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,11,5,2008,66,0.666667,Neutral,13,30,44,01dbf56d-681b-47e1-917a-af746d6f906d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,5,2008,64,0.656250,Neutral,18,34,42,01dbf56d-681b-47e1-917a-af746d6f906d
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,11,5,2008,91,0.780220,Away,14,40,71,3a1dcea9-5dc4-4468-af40-bbdeb727914b
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,11,5,2008,89,1.146067,Home,26,48,102,3a1dcea9-5dc4-4468-af40-bbdeb727914b
5,Richmond,"L, 63-80",2007-11-06,Memphis,V,200,63,0.801,76,24,...,11,6,2008,79,0.797468,Away,16,41,63,4ced5612-488a-42dd-ac40-cd016d5f7644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,4,1,2023,64,1.109375,Away,14,30,71,62792664-cb9d-46f6-a0ae-da5b4eac020a
171965,Connecticut,"W, 72-59",2023-04-01,Miami (Fla.),H,195,72,1.122,64,28,...,4,1,2023,64,1.125000,Home,19,31,72,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,4,1,2023,65,0.907692,Away,13,42,59,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,4,3,2023,69,0.855072,Away,13,36,59,9ba510b2-bd9b-4180-9602-0042dc6bd324


There are 13 observations where the counterpart for the teams opponent is missing

In [555]:
missing_df = statline_df.loc[statline_df['UUID'].isna()]
missing_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID
26416,Montana State,"L, 61-71",2010-01-18,Central Arkansas,V,200,61,0.966,63,23,...,1,18,2010,63,0.968254,Away,17,28,61,None
84460,Lehigh,"L, 55-61",2015-02-28,Colgate,V,77,0,0.000,0,0,...,2,28,2015,0,0.000000,Away,0,0,0,None
147984,Saint Peter`s,"L, 65-78",2021-02-27,Rider,H,7,0,0.000,0,0,...,2,27,2021,1,0.000000,Home,0,1,0,None
148020,South Alabama,"L, 73-82",2021-02-27,Georgia State,H,37,9,0.818,11,3,...,2,27,2021,11,0.818182,Home,0,1,9,None
148080,Charlotte,"L, 62-77",2021-02-28,UTEP,V,15,4,1.000,2,2,...,2,28,2021,4,1.000000,Away,2,2,4,None
160332,Albany,"L, 62-67",2022-11-07,Towson,V,194,62,0.932,67,19,...,11,7,2023,67,0.925373,Away,9,22,62,None
160378,Houston Christian,"L, 66-77",2022-11-07,Florida International,V,102,21,0.519,40,8,...,11,7,2023,40,0.525000,Away,5,19,21,None


In [556]:
not_missing = statline_df.loc[~statline_df['UUID'].isna()]
non_duplicated = not_missing[~not_missing.duplicated('UUID', keep=False)]
non_duplicated

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID
21576,Miami (Fla.),"W, 74-54",2009-11-20,Tulane,N,200,67,0.913,73,26,...,11,20,2010,74,0.905405,Neutral,23,41,67,ba255257-2baa-401a-830d-140d7717a7b6
143865,Sam Houston State,"W, 70-52",2021-01-06,Southeastern Louisiana,V,197,70,0.989,71,26,...,1,6,2021,71,0.985915,Away,12,31,70,bcb54ca1-97f4-4832-8eb2-30c5a5c18743
144347,Sam Houston State,"W, 96-71",2021-01-13,Lamar,H,193,96,1.213,79,36,...,1,13,2021,79,1.215190,Home,22,34,96,1dfcc023-7002-4f6c-8bea-e09e162eb8b4
144767,Sam Houston State,"W, 87-80",2021-01-16,Houston Christian,H,195,87,1.178,74,29,...,1,16,2021,74,1.175676,Home,17,34,87,89f466d6-25dc-4c67-abc3-6485914e458d
144914,Sam Houston State,"W, 64-57",2021-01-20,Abilene Christian,H,195,64,0.965,66,21,...,1,20,2021,67,0.955224,Home,15,36,64,f7022066-ee65-4692-95fa-de03083de1ee
146267,Sam Houston State,"W, 78-71",2021-02-06,Nicholls State,H,195,78,0.946,82,28,...,2,6,2021,83,0.939759,Home,20,40,78,3e5cb6be-7b85-4985-af65-1dd2294ba8c4


In [557]:
missing_obs = set(missing_df['Obs UUID'])
non_duplicated_obs = set(non_duplicated['Obs UUID'])
removal_obs = missing_obs.union(non_duplicated_obs)
statline_df = statline_df.loc[~statline_df['Obs UUID'].isin(removal_obs)]
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Month,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,11,5,2008,66,0.666667,Neutral,13,30,44,01dbf56d-681b-47e1-917a-af746d6f906d
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,11,5,2008,64,0.656250,Neutral,18,34,42,01dbf56d-681b-47e1-917a-af746d6f906d
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,11,5,2008,91,0.780220,Away,14,40,71,3a1dcea9-5dc4-4468-af40-bbdeb727914b
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,11,5,2008,89,1.146067,Home,26,48,102,3a1dcea9-5dc4-4468-af40-bbdeb727914b
5,Richmond,"L, 63-80",2007-11-06,Memphis,V,200,63,0.801,76,24,...,11,6,2008,79,0.797468,Away,16,41,63,4ced5612-488a-42dd-ac40-cd016d5f7644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,4,1,2023,64,1.109375,Away,14,30,71,62792664-cb9d-46f6-a0ae-da5b4eac020a
171965,Connecticut,"W, 72-59",2023-04-01,Miami (Fla.),H,195,72,1.122,64,28,...,4,1,2023,64,1.125000,Home,19,31,72,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,4,1,2023,65,0.907692,Away,13,42,59,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,4,3,2023,69,0.855072,Away,13,36,59,9ba510b2-bd9b-4180-9602-0042dc6bd324


### Combine Rows for Same Game

We are assigning each observation a random value. When the observations for a common game is combined, one team will be "Team 1" and the other will be "Team 2", the random value is so there is no correlation between which team gets which assignment.

In [558]:
statline_df['Random Value'] = [random.random() for _ in range(len(statline_df))]
statline_df

C:\Users\Colin\AppData\Local\Temp\ipykernel_24104\3018805021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statline_df['Random Value'] = [random.random() for _ in range(len(statline_df))]


,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID,Random Value
0,Richmond,"W, 44-42",2007-11-05,Maine,N,200,44,0.663,65,17,...,5,2008,66,0.666667,Neutral,13,30,44,01dbf56d-681b-47e1-917a-af746d6f906d,0.039249
1,Maine,"L, 42-44",2007-11-05,Richmond,N,200,42,0.652,65,19,...,5,2008,64,0.656250,Neutral,18,34,42,01dbf56d-681b-47e1-917a-af746d6f906d,0.077527
2,Tennessee-Martin,"L, 71-102",2007-11-05,Memphis,V,200,71,0.785,90,24,...,5,2008,91,0.780220,Away,14,40,71,3a1dcea9-5dc4-4468-af40-bbdeb727914b,0.480556
3,Memphis,"W, 102-71",2007-11-05,Tennessee-Martin,H,200,102,1.149,90,37,...,5,2008,89,1.146067,Home,26,48,102,3a1dcea9-5dc4-4468-af40-bbdeb727914b,0.052867
5,Richmond,"L, 63-80",2007-11-06,Memphis,V,200,63,0.801,76,24,...,6,2008,79,0.797468,Away,16,41,63,4ced5612-488a-42dd-ac40-cd016d5f7644,0.123992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171967,FAU,"L, 71-72",2023-04-01,San Diego State,V,196,71,1.110,64,23,...,1,2023,64,1.109375,Away,14,30,71,62792664-cb9d-46f6-a0ae-da5b4eac020a,0.885114
171965,Connecticut,"W, 72-59",2023-04-01,Miami (Fla.),H,195,72,1.122,64,28,...,1,2023,64,1.125000,Home,19,31,72,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156,0.739757
171968,Miami (Fla.),"L, 59-72",2023-04-01,Connecticut,V,195,59,0.912,64,20,...,1,2023,65,0.907692,Away,13,42,59,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156,0.024116
171969,San Diego State,"L, 59-76",2023-04-03,Connecticut,V,193,59,0.861,68,19,...,3,2023,69,0.855072,Away,13,36,59,9ba510b2-bd9b-4180-9602-0042dc6bd324,0.144267


In [559]:
statline_df = statline_df.sort_values(by=['UUID', 'Random Value'], kind='mergesort')
statline_df

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID,Random Value
128991,Houston Christian,"W, 118-111",2019-03-06,Incarnate Word,H,200,118,1.457,80,39,...,6,2019,82,1.439024,Home,29,42,118,000021aa-83c6-429a-a047-baca985f515d,0.315116
129059,Incarnate Word,"L, 111-118",2019-03-06,Houston Christian,V,200,111,1.381,80,40,...,6,2019,81,1.370370,Away,30,42,111,000021aa-83c6-429a-a047-baca985f515d,0.486580
89510,Wyoming,"L, 89-94",2015-12-22,Houston,N,250,89,0.948,93,22,...,22,2016,95,0.936842,Neutral,13,29,89,0001dbdc-176a-439b-abe8-46a87a975fbb,0.226850
89482,Houston,"W, 94-89",2015-12-22,Wyoming,N,250,94,1.023,93,33,...,22,2016,92,1.021739,Neutral,25,46,94,0001dbdc-176a-439b-abe8-46a87a975fbb,0.503495
73283,Southeastern Louisiana,"L, 83-88",2014-02-27,Northwestern State,H,200,83,1.129,74,28,...,27,2014,74,1.121622,Home,17,34,83,00024074-4938-447d-a655-35871cdf4ce9,0.012189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61669,Hofstra,"L, 50-79",2013-02-20,George Mason,H,200,50,0.766,64,15,...,20,2013,66,0.757576,Home,13,38,50,ffff2816-3b04-4ddc-98b8-2d0704681236,0.120184
11636,Saint Mary`s,"L, 62-75",2008-11-27,UTEP,N,200,62,0.811,76,22,...,27,2009,77,0.805195,Neutral,19,37,62,ffff7141-cda1-4d07-b7bc-261d42464517,0.397966
11644,UTEP,"W, 75-62",2008-11-27,Saint Mary`s,N,200,75,0.992,76,22,...,27,2009,76,0.986842,Neutral,20,40,75,ffff7141-cda1-4d07-b7bc-261d42464517,0.457869
75946,Navy,"L, 44-68",2014-11-22,Northeastern,N,200,44,0.803,58,19,...,22,2015,55,0.800000,Neutral,15,30,44,ffffcc9d-968f-4ae4-9103-ffe33e17f791,0.160574


In [560]:
team1_statlines = statline_df.drop_duplicates(subset='UUID', keep='first')
team1_statlines

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID,Random Value
128991,Houston Christian,"W, 118-111",2019-03-06,Incarnate Word,H,200,118,1.457,80,39,...,6,2019,82,1.439024,Home,29,42,118,000021aa-83c6-429a-a047-baca985f515d,0.315116
89510,Wyoming,"L, 89-94",2015-12-22,Houston,N,250,89,0.948,93,22,...,22,2016,95,0.936842,Neutral,13,29,89,0001dbdc-176a-439b-abe8-46a87a975fbb,0.226850
73283,Southeastern Louisiana,"L, 83-88",2014-02-27,Northwestern State,H,200,83,1.129,74,28,...,27,2014,74,1.121622,Home,17,34,83,00024074-4938-447d-a655-35871cdf4ce9,0.012189
28210,Cal State Fullerton,"L, 59-76",2010-02-06,UC Riverside,H,200,59,0.963,60,18,...,6,2010,62,0.951613,Home,12,30,59,0002c799-1ae2-44cc-bdd5-9179133a967a,0.019475
96221,James Madison,"L, 64-79",2016-03-05,William & Mary,N,200,64,0.920,69,22,...,5,2016,70,0.914286,Neutral,13,30,64,000326a3-3879-41ef-bae3-2ecf69461e11,0.236472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127269,Manhattan,"W, 64-60",2019-02-15,Niagara,V,200,64,1.110,59,21,...,15,2019,58,1.103448,Away,11,26,64,fffe6833-bf07-4a84-8ec2-580337b888b6,0.209497
147649,Stephen F. Austin,"W, 83-57",2021-02-24,Northwestern State,V,196,83,1.187,70,34,...,24,2021,70,1.185714,Away,28,47,83,fffe6e3d-dbd4-49d2-bb2c-396a3a1ab9e7,0.100320
61631,George Mason,"W, 79-50",2013-02-20,Hofstra,V,200,79,1.253,64,28,...,20,2013,63,1.253968,Away,20,38,79,ffff2816-3b04-4ddc-98b8-2d0704681236,0.041879
11636,Saint Mary`s,"L, 62-75",2008-11-27,UTEP,N,200,62,0.811,76,22,...,27,2009,77,0.805195,Neutral,19,37,62,ffff7141-cda1-4d07-b7bc-261d42464517,0.397966


In [561]:
team1_ids = team1_statlines['Obs UUID'].unique()
team2_statlines = statline_df.loc[~statline_df['Obs UUID'].isin(team1_ids)]
team2_statlines

,Team,Res,Date,Opp,Loc,MIN,PTS,O-PPP,Pace,FGM,...,Day,Season,Calculated Possessions,Points Per Possession,Location,2FM,2FA,Calculated Points,UUID,Random Value
129059,Incarnate Word,"L, 111-118",2019-03-06,Houston Christian,V,200,111,1.381,80,40,...,6,2019,81,1.370370,Away,30,42,111,000021aa-83c6-429a-a047-baca985f515d,0.486580
89482,Houston,"W, 94-89",2015-12-22,Wyoming,N,250,94,1.023,93,33,...,22,2016,92,1.021739,Neutral,25,46,94,0001dbdc-176a-439b-abe8-46a87a975fbb,0.503495
73248,Northwestern State,"W, 88-83",2014-02-27,Southeastern Louisiana,V,200,88,1.189,74,34,...,27,2014,74,1.189189,Away,29,43,88,00024074-4938-447d-a655-35871cdf4ce9,0.466522
28017,UC Riverside,"W, 76-59",2010-02-06,Cal State Fullerton,V,200,76,1.257,60,28,...,6,2010,61,1.245902,Away,22,47,76,0002c799-1ae2-44cc-bdd5-9179133a967a,0.673211
96080,William & Mary,"W, 79-64",2016-03-05,James Madison,N,200,79,1.164,69,25,...,5,2016,68,1.161765,Neutral,19,30,79,000326a3-3879-41ef-bae3-2ecf69461e11,0.680705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127263,Niagara,"L, 60-64",2019-02-15,Manhattan,H,200,60,1.007,59,22,...,15,2019,60,1.000000,Home,16,31,60,fffe6833-bf07-4a84-8ec2-580337b888b6,0.694018
147642,Northwestern State,"L, 57-83",2021-02-24,Stephen F. Austin,H,194,57,0.805,70,15,...,24,2021,71,0.802817,Home,9,23,57,fffe6e3d-dbd4-49d2-bb2c-396a3a1ab9e7,0.275129
61669,Hofstra,"L, 50-79",2013-02-20,George Mason,H,200,50,0.766,64,15,...,20,2013,66,0.757576,Home,13,38,50,ffff2816-3b04-4ddc-98b8-2d0704681236,0.120184
11644,UTEP,"W, 75-62",2008-11-27,Saint Mary`s,N,200,75,0.992,76,22,...,27,2009,76,0.986842,Neutral,20,40,75,ffff7141-cda1-4d07-b7bc-261d42464517,0.457869


In [562]:
statline_df = team1_statlines.merge(team2_statlines, how='outer', on='UUID', suffixes=('_1', '_2'))
statline_df

,Team_1,Res_1,Date_1,Opp_1,Loc_1,MIN_1,PTS_1,O-PPP_1,Pace_1,FGM_1,...,Month_2,Day_2,Season_2,Calculated Possessions_2,Points Per Possession_2,Location_2,2FM_2,2FA_2,Calculated Points_2,Random Value_2
0,Houston Christian,"W, 118-111",2019-03-06,Incarnate Word,H,200,118,1.457,80,39,...,3,6,2019,81,1.370370,Away,30,42,111,0.486580
1,Wyoming,"L, 89-94",2015-12-22,Houston,N,250,89,0.948,93,22,...,12,22,2016,92,1.021739,Neutral,25,46,94,0.503495
2,Southeastern Louisiana,"L, 83-88",2014-02-27,Northwestern State,H,200,83,1.129,74,28,...,2,27,2014,74,1.189189,Away,29,43,88,0.466522
3,Cal State Fullerton,"L, 59-76",2010-02-06,UC Riverside,H,200,59,0.963,60,18,...,2,6,2010,61,1.245902,Away,22,47,76,0.673211
4,James Madison,"L, 64-79",2016-03-05,William & Mary,N,200,64,0.920,69,22,...,3,5,2016,68,1.161765,Neutral,19,30,79,0.680705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85198,Manhattan,"W, 64-60",2019-02-15,Niagara,V,200,64,1.110,59,21,...,2,15,2019,60,1.000000,Home,16,31,60,0.694018
85199,Stephen F. Austin,"W, 83-57",2021-02-24,Northwestern State,V,196,83,1.187,70,34,...,2,24,2021,71,0.802817,Home,9,23,57,0.275129
85200,George Mason,"W, 79-50",2013-02-20,Hofstra,V,200,79,1.253,64,28,...,2,20,2013,66,0.757576,Home,13,38,50,0.120184
85201,Saint Mary`s,"L, 62-75",2008-11-27,UTEP,N,200,62,0.811,76,22,...,11,27,2009,76,0.986842,Neutral,20,40,75,0.457869


There are about 140 games where the data are missing, only NC State has had a game where the team has scored less than 33 points since 2007 

In [563]:
statline_df = statline_df.loc[((statline_df['Calculated Points_1'] > 32) | (statline_df['Team_1'] == 'North Carolina State')) &
                              ((statline_df['Calculated Points_2'] > 32) | (statline_df['Team_2'] == 'North Carolina State'))]
statline_df

,Team_1,Res_1,Date_1,Opp_1,Loc_1,MIN_1,PTS_1,O-PPP_1,Pace_1,FGM_1,...,Month_2,Day_2,Season_2,Calculated Possessions_2,Points Per Possession_2,Location_2,2FM_2,2FA_2,Calculated Points_2,Random Value_2
0,Houston Christian,"W, 118-111",2019-03-06,Incarnate Word,H,200,118,1.457,80,39,...,3,6,2019,81,1.370370,Away,30,42,111,0.486580
1,Wyoming,"L, 89-94",2015-12-22,Houston,N,250,89,0.948,93,22,...,12,22,2016,92,1.021739,Neutral,25,46,94,0.503495
2,Southeastern Louisiana,"L, 83-88",2014-02-27,Northwestern State,H,200,83,1.129,74,28,...,2,27,2014,74,1.189189,Away,29,43,88,0.466522
3,Cal State Fullerton,"L, 59-76",2010-02-06,UC Riverside,H,200,59,0.963,60,18,...,2,6,2010,61,1.245902,Away,22,47,76,0.673211
4,James Madison,"L, 64-79",2016-03-05,William & Mary,N,200,64,0.920,69,22,...,3,5,2016,68,1.161765,Neutral,19,30,79,0.680705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85198,Manhattan,"W, 64-60",2019-02-15,Niagara,V,200,64,1.110,59,21,...,2,15,2019,60,1.000000,Home,16,31,60,0.694018
85199,Stephen F. Austin,"W, 83-57",2021-02-24,Northwestern State,V,196,83,1.187,70,34,...,2,24,2021,71,0.802817,Home,9,23,57,0.275129
85200,George Mason,"W, 79-50",2013-02-20,Hofstra,V,200,79,1.253,64,28,...,2,20,2013,66,0.757576,Home,13,38,50,0.120184
85201,Saint Mary`s,"L, 62-75",2008-11-27,UTEP,N,200,62,0.811,76,22,...,11,27,2009,76,0.986842,Neutral,20,40,75,0.457869


Since we know the Res column is often incorrect, we are recalculating which team won based off the final score

In [564]:
statline_df['Victory_1'] = statline_df.apply(lambda r: 1 if r['Calculated Points_1'] > r['Calculated Points_2'] else 0, axis=1)
statline_df['Victory_2'] = statline_df.apply(lambda r: 1 if r['Calculated Points_1'] < r['Calculated Points_2'] else 0, axis=1)

C:\Users\Colin\AppData\Local\Temp\ipykernel_24104\597576929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statline_df['Victory_1'] = statline_df.apply(lambda r: 1 if r['Calculated Points_1'] > r['Calculated Points_2'] else 0, axis=1)
C:\Users\Colin\AppData\Local\Temp\ipykernel_24104\597576929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statline_df['Victory_2'] = statline_df.apply(lambda r: 1 if r['Calculated Points_1'] < r['Calculated Points_2'] else 0, axis=1)


### Error Checking

In [565]:
pd.Series(statline_df['Victory_1'] == statline_df['Victory_2']).any()

True

In [566]:
any((statline_df['Victory_1'] == 1) & (statline_df['Calculated Points_1'] < statline_df['Calculated Points_2']))

False

In [567]:
statline_df.loc[(statline_df['Victory_1'] == 1) & (statline_df['Calculated Points_1'] < statline_df['Calculated Points_2'])][['Team_1', 'Team_2', 'Date_1', 'Season_1', 'Res_1', 'PTS_1', 'PTS_2', 'Calculated Points_1', 'Calculated Points_2']]

,Team_1,Team_2,Date_1,Season_1,Res_1,PTS_1,PTS_2,Calculated Points_1,Calculated Points_2


In [568]:
any((statline_df['Victory_2'] == 1) & (statline_df['Calculated Points_2'] < statline_df['Calculated Points_1']))

False

In [569]:
statline_df.loc[(statline_df['Victory_2'] == 1) & (statline_df['Calculated Points_2'] < statline_df['Calculated Points_1'])][['Team_1', 'Team_2', 'Date_1', 'Season_1', 'Res_1', 'PTS_1', 'PTS_2', 'Calculated Points_1', 'Calculated Points_2']]

,Team_1,Team_2,Date_1,Season_1,Res_1,PTS_1,PTS_2,Calculated Points_1,Calculated Points_2


In [570]:
pd.Series(statline_df['Date_1'] != statline_df['Date_2']).any()

False

In [571]:
pd.Series(statline_df['Year_1'] != statline_df['Year_2']).any()

False

In [572]:
pd.Series(statline_df['Month_1'] != statline_df['Month_2']).any()

False

In [573]:
pd.Series(statline_df['Day_1'] != statline_df['Day_2']).any()

False

In [574]:
pd.Series(statline_df['Season_1'] != statline_df['Season_2']).any()

False

In [575]:
pd.Series((statline_df['Team_1'] != statline_df['Opp_2']) | (statline_df['Team_2'] != statline_df['Opp_1'])).any()

False

In [576]:
pd.Series((statline_df['PTS_1'] != statline_df['Calculated Points_1']) | (statline_df['PTS_2'] != statline_df['Calculated Points_2'])).any()

False

### Drop Redundant Columns and Rename Columns

In [577]:
statline_df = statline_df.drop(columns=['Res_1', 'Opp_1', 'Loc_1', 'Obs UUID_1', 'Random Value_1', 'Res_2', 'Date_2', 'Opp_2', 'Loc_2',
                                        'Obs UUID_2', 'Victory_2', 'Year_2', 'Month_2', 'Day_2', 'Season_2', 'Location_2', 
                                        'Random Value_2'])

statline_df = statline_df.rename(columns={'Date_1': 'Date',
                                          'Victory_1': 'Team_1_Victory',
                                          'Year_1': 'Year',
                                          'Month_1': 'Month',
                                          'Day_1': 'Day',
                                          'Season_1': 'Season',
                                          'Calculated Points_1': 'Points_1',
                                          'Calculated Points_2': 'Points_2',
                                          'Calculated Possessions_1': 'Possessions_1',
                                          'Calculated Possessions_2': 'Possessions_2',
                                          'Location_1': 'Team_1_Location'})
statline_df

,Team_1,Date,MIN_1,PTS_1,O-PPP_1,Pace_1,FGM_1,FGA_1,3FM_1,3FA_1,...,AST_2,STL_2,BLK_2,TO_2,F_2,Possessions_2,Points Per Possession_2,2FM_2,2FA_2,Points_2
0,Houston Christian,2019-03-06,200,118,1.457,80,39,62,10,20,...,22,2,2,12,30,81,1.370370,30,42,111
1,Wyoming,2015-12-22,250,89,0.948,93,22,57,9,28,...,12,10,2,12,35,92,1.021739,25,46,94
2,Southeastern Louisiana,2014-02-27,200,83,1.129,74,28,57,11,23,...,16,5,2,13,16,74,1.189189,29,43,88
3,Cal State Fullerton,2010-02-06,200,59,0.963,60,18,44,6,14,...,12,4,0,9,21,61,1.245902,22,47,76
4,James Madison,2016-03-05,200,64,0.920,69,22,62,9,32,...,15,7,1,10,18,68,1.161765,19,30,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85198,Manhattan,2019-02-15,200,64,1.110,59,21,47,10,21,...,12,2,6,10,17,60,1.000000,16,31,60
85199,Stephen F. Austin,2021-02-24,196,83,1.187,70,34,65,6,18,...,7,3,3,18,19,71,0.802817,9,23,57
85200,George Mason,2013-02-20,200,79,1.253,64,28,52,8,14,...,6,5,4,11,18,66,0.757576,13,38,50
85201,Saint Mary`s,2008-11-27,200,62,0.811,76,22,61,3,24,...,11,4,2,18,29,76,0.986842,20,40,75


### Reorder Columns

In [578]:
statline_df = statline_df[['UUID', 'Date', 'Year', 'Month', 'Day', 'Season', 'Team_1_Location', 'Team_1_Victory',
                           'Team_1', 'MIN_1', 'Points_1', 'Points Per Possession_1', 'Possessions_1', 'FGM_1', 'FGA_1',  '2FM_1', '2FA_1', 
                           '3FM_1', '3FA_1', 'FTM_1', 'FTA_1', 'OREB_1', 'TmRb_1', 'REB_1', 'AST_1', 'STL_1', 'BLK_1', 'TO_1', 'F_1',
                           'Team_2', 'MIN_2', 'Points_2', 'Points Per Possession_2', 'Possessions_2', 'FGM_2', 'FGA_2',  '2FM_2', '2FA_2',
                           '3FM_2', '3FA_2', 'FTM_2', 'FTA_2', 'OREB_2', 'TmRb_2', 'REB_2', 'AST_2', 'STL_2', 'BLK_2', 'TO_2', 'F_2']]
statline_df = statline_df.sort_values(by='Date').reset_index(drop=True)
statline_df

,UUID,Date,Year,Month,Day,Season,Team_1_Location,Team_1_Victory,Team_1,MIN_1,...,FTM_2,FTA_2,OREB_2,TmRb_2,REB_2,AST_2,STL_2,BLK_2,TO_2,F_2
0,01dbf56d-681b-47e1-917a-af746d6f906d,2007-11-05,2007,11,5,2008,Neutral,1,Richmond,200,...,3,5,11,0,39,9,9,1,28,12
1,3a1dcea9-5dc4-4468-af40-bbdeb727914b,2007-11-05,2007,11,5,2008,Home,1,Memphis,200,...,13,22,15,0,41,16,5,2,18,22
2,a83307d5-9410-475f-b95a-f054f5cb2111,2007-11-06,2007,11,6,2008,Neutral,1,Gardner-Webb,200,...,17,26,10,0,30,9,4,8,27,24
3,4ced5612-488a-42dd-ac40-cd016d5f7644,2007-11-06,2007,11,6,2008,Away,0,Richmond,200,...,21,32,11,0,36,15,13,10,17,18
4,c7635057-16be-4f56-ad92-102d30d33caf,2007-11-06,2007,11,6,2008,Home,1,Kentucky,200,...,17,21,11,0,25,5,5,4,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85058,a2394567-37c5-48d1-8086-9a1c4fc4691c,2023-03-28,2023,3,28,2023,Away,0,Utah Valley,221,...,17,22,11,0,42,14,9,4,10,23
85059,192dafc9-e857-4764-9e8e-3a04af1bb73c,2023-03-30,2023,3,30,2023,Home,1,North Texas,196,...,24,31,9,0,27,8,7,2,11,14
85060,62792664-cb9d-46f6-a0ae-da5b4eac020a,2023-04-01,2023,4,1,2023,Home,1,San Diego State,195,...,16,21,7,0,31,6,6,2,9,17
85061,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156,2023-04-01,2023,4,1,2023,Away,0,Miami (Fla.),195,...,7,13,13,0,40,19,5,5,14,11


In [579]:
statline_df.to_csv('Historical Data/Team Statlines/team-statlines-2007-23.csv', index=False)

# Basic Calculations

In [580]:
statline_df = pd.read_csv('Historical Data/Team Statlines/team-statlines-2007-23.csv')
statline_df

,UUID,Date,Year,Month,Day,Season,Team_1_Location,Team_1_Victory,Team_1,MIN_1,...,FTM_2,FTA_2,OREB_2,TmRb_2,REB_2,AST_2,STL_2,BLK_2,TO_2,F_2
0,01dbf56d-681b-47e1-917a-af746d6f906d,2007-11-05,2007,11,5,2008,Neutral,1,Richmond,200,...,3,5,11,0,39,9,9,1,28,12
1,3a1dcea9-5dc4-4468-af40-bbdeb727914b,2007-11-05,2007,11,5,2008,Home,1,Memphis,200,...,13,22,15,0,41,16,5,2,18,22
2,a83307d5-9410-475f-b95a-f054f5cb2111,2007-11-06,2007,11,6,2008,Neutral,1,Gardner-Webb,200,...,17,26,10,0,30,9,4,8,27,24
3,4ced5612-488a-42dd-ac40-cd016d5f7644,2007-11-06,2007,11,6,2008,Away,0,Richmond,200,...,21,32,11,0,36,15,13,10,17,18
4,c7635057-16be-4f56-ad92-102d30d33caf,2007-11-06,2007,11,6,2008,Home,1,Kentucky,200,...,17,21,11,0,25,5,5,4,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85058,a2394567-37c5-48d1-8086-9a1c4fc4691c,2023-03-28,2023,3,28,2023,Away,0,Utah Valley,221,...,17,22,11,0,42,14,9,4,10,23
85059,192dafc9-e857-4764-9e8e-3a04af1bb73c,2023-03-30,2023,3,30,2023,Home,1,North Texas,196,...,24,31,9,0,27,8,7,2,11,14
85060,62792664-cb9d-46f6-a0ae-da5b4eac020a,2023-04-01,2023,4,1,2023,Home,1,San Diego State,195,...,16,21,7,0,31,6,6,2,9,17
85061,7ed7115c-cb3a-43d4-9aff-d05ffa5b0156,2023-04-01,2023,4,1,2023,Away,0,Miami (Fla.),195,...,7,13,13,0,40,19,5,5,14,11
